## In this lab session we will implement two algorithms (an algorithm for sorting and one for detecting communities) each in two different ways: a naive, less efficient way and a better, faster way

In [41]:
from collections import defaultdict
from itertools import count

import time
import copy
import numpy as np

import matplotlib.pyplot as plt
import plotly.offline as py
from plotly.offline import init_notebook_mode
import plotly.graph_objs as go
init_notebook_mode(connected=True)

import networkx as nx

### Let's start with the sorting algorithm
We will create multiple data sets of integers we want to sort <br>
Each data set bigger than the previous one <br>
These different sized data sets will be use to emperically determine the complexity of the algorithm

In [42]:
def apply_and_time_sorting(datasets, sort_function):
    timings = []
    for dataset in datasets:
        start = time.time()
        sort_function(dataset)
        end = time.time()
        time_difference = end - start
        timings.append(time_difference)
    return timings

In [43]:
datasets = []
for data_size in [5000,10000,20000]:
    datasets.append(np.random.permutation(data_size))

### Here we will implement the naive insertion sorting algorithm 
It's an algorithm that sorts an array in-place by iterating over the elements and inserting the current element
in the part of the array that is already sorted

```python
[1,11,13,7,4,8,0]
```

If the current element is 7, then the part of the algorithm before 7 is already sorted by the algorithm <br>
It then tries to find the position to insert the element 7 in the part of the array that is already sorted <br>
Resulting in

```python
[1,7,11,13,4,8,0]
```

The next element to be sorted is 4

In [44]:
def naivesort(data):
    for i, element in enumerate(data):
        j = i
        while j>0 and element<data[j-1]: #iterate back into the array as long as the current element is smaller
            data[j] = data[j-1] #put the bigger element in the position of the current one
            j -= 1
        data[j] = element # store the current element in its sorted position

In [45]:
timings = apply_and_time_sorting(datasets, naivesort)
print(timings)

[3.3475136756896973, 13.507608413696289, 54.321414947509766]


### Let's compare this with the quicksort algorithm that was discussed in class

In [46]:
datasets = []
for data_size in [5000,10000,20000]:
    datasets.append(np.random.permutation(data_size))

In [47]:
def swap(l, i, j):
    tmp = l[j]
    l[j] = l[i]
    l[i] = tmp

def quicksort_helper(l, starti, endi):
    #set_trace()
    if starti >= endi:
        return
    if endi-starti+1<=100:
        naivesort(l)
    else:
        middlei = int(0.5*(starti+endi))
        #first make sure the start, middle and end element are in sorted order
        if (l[middlei]<l[starti]):
            swap(l,starti, middlei)
        if (l[endi]<l[starti]):
            swap(l, starti, endi)
        if (l[endi]<l[middlei]):
            swap(l, middlei, endi)
        pivoti = middlei
        
        #put the pivot element just before the last 
        swap(l, pivoti, endi-1)
        pivot = l[endi-1]
        lefti = starti+1
        righti = endi-2
        
        #now make sure the elements are partially sorted,
        #i.e. elements in the first half should be smaller than the pivot element
        #the elements in the second half should be larger than the pivot element
        while True:
            while l[lefti]<pivot: #move the cursor from left to right until you encounter an element larger than the pivot
                lefti += 1
            while pivot<l[righti]: #move the cursor from right to left until you encounter an element larger than the pivot
                righti -= 1
            if lefti < righti: #the right element should be smaller than the left element, swap them 
                swap(l, lefti, righti)
            else:
                break
                
        pivoti = lefti
        swap(l, pivoti, endi-1)
        #recursive sort the first half of the array
        quicksort_helper(l, starti, pivoti-1)
        #recursive sort the second half of the array
        quicksort_helper(l, pivoti+1, endi)
        
def quicksort(l):
    quicksort_helper(l, 0, len(l)-1)

In [48]:
timings = apply_and_time_sorting(datasets, quicksort)
print(timings)

[2.248666524887085, 12.91700005531311, 19.3950617313385]


## The next algorithm we will investigate is the community detection algorithm: the Louvain method

### As a simple example we will take the social network of a karate club collected by Zachary, the members have a link between them in case they interacted with each other outside the club. The goal is to find the communities that exist in the club

In [49]:
file = "./data/karate_edges_78.txt"

### Let's first create the adjacency matrix for the social network

In [54]:
def create_adjacency_matrix(file):
    node_neighbors = defaultdict(list)

    with open(file, "r") as fin:
        for line in fin:
            node_1, node_2 = line.strip().split('\t')
            node_1 = int(node_1)-1 #offset to make the integers start from zero
            node_2 = int(node_2)-1
            node_neighbors[node_1].append(node_2)

    number_of_nodes = len(node_neighbors)
    A = np.zeros((number_of_nodes, number_of_nodes))

    for node, neighbors in node_neighbors.items():
        for neighbor in neighbors:
            A[node, neighbor] = 1

    return A

In [55]:
A = create_adjacency_matrix(file)

### Compute the total sum of edges in the adjacency matrix A (divided by 2)
Hint: use np.sum

In [68]:
m = 0.5*np.sum(np.sum(A))

### Let's now define a function to compute the modularity of the network given the adjacency matrix A, the communities and the total sum of edge weights

In [69]:
def compute_modularity(A, communities, m):
    """
    A: dense square matrix, nonzero element on row i and column j indicating 
        a weighted connection between vertex i and j
    communities: vector of communities, on position i the community for node i, so the length of the list equals 
        the number of nodes in the graph
    m: an integer that equals the sum of all edges in the graph divided by two
    """
    nrows, ncols = A.shape
    assert nrows == ncols
    nvertices = nrows

    modularity = 0.0
    for vertexi in range(nvertices):
        for vertexj in range(nvertices):
            ki = np.sum(A[vertexi,:])
            kj = np.sum(A[vertexj,:])
            if communities[vertexi] == communities[vertexj]:
                modularity += A[vertexi, vertexj] - (ki*kj)/(2*m)

    return modularity/(2*m)

### Compute the modularity putting every vertex/node in its own community

The community vector has the same length as the number of vertices/nodes in the graph <br>
Position i in the community vector defines the community of vertex/node i <br>
e.g. [0,0,1,1,1] means the first two vertices belong the community 0 and the last three to community 1

In [70]:
def initialize_communities(n_vertices):
    return np.arange(0,n_vertices) # use np.arange to create a vector from 0 to n_vertices-1
    
communities = initialize_communities(A.shape[0])

### Now compute the modularity for the given adjancency matrix, the initial communities and the total sum of edges

In [71]:
initial_modularity = compute_modularity(A, communities, m)

In [74]:
assert initial_modularity == -0.04980276134122286

### Now we will implement the first phase of the Louvain method but using a naive, inefficient way of computing the modularity difference

Let's first define some helper function to perform the local search (i.e. the first phase) <br>

The first helper function let's us compute the difference in modularity when we move a node from its current
community into another community <br>

We will (on purpose) compute this in a very naive inefficient way

In [81]:
def modularity_difference(A, communities, m, modularity, vertex, community):
    """
    A: the adjacency matrix of the graph
    communities: vector of communities, on position i the community for node i, so the length of the list equals 
        the number of nodes in the graph 
    m: an integer that equals the sum of all edges in the graph divided by two
    modularity: the modularity of the graph A with communities defined by the variable communities
    vertex: the index of the vertex/node you want to compute the modularity difference for when moving it to community
    community: the community to which you want move the vertex
    """
    #save community of node
    original_community = communities[vertex]
    #change community
    communities[vertex] = community
    #compute new modularity
    new_modularity = compute_modularity(A, communities, m)
    #print(f"{nodei}, {communityi}, {self.modularity}, {new_modularity}")
    modularity_difference = new_modularity-modularity
    #restore community
    communities[vertex] = original_community
    return modularity_difference

### Now use the modularity_difference function to iterate over all communities and see which community for the given vertex gives the biggest increase in modularity

In [82]:
def find_best_community(A, communities, m, modularity, vertex):
    """
    A: the adjacency matrix of the graph
    communities: vector of communities, on position i the community for node i, so the length of the list equals 
        the number of nodes in the graph 
    m: an integer that equals the sum of all edges in the graph divided by two
    modularity: the modularity of the graph A with communities defined by the variable communities
    vertex: the index of the vertex/node for which you want to find the best community
    """    
    unique_communities = np.unique(communities)
    max_modularity_difference = 0
    best_community = communities[vertex]
    for community in unique_communities:
        modularity_diff = modularity_difference(A, communities, m, modularity, vertex, community)
        print(f"modularity difference when moving {vertex} to community {community}: {modularity_diff}")
        if modularity_diff > max_modularity_difference:
            max_modularity_difference = modularity_diff
            best_community = community
    return best_community, max_modularity_difference

### We will now use the find_best_community function to keep iterating over all nodes until we can't increase the modularity anymore

In [ ]:
def local_search(A, communities, m, modularity, seed=11):
    nvertices = len(communities)
    if seed>0:
        vertex_order = np.random.RandomState(seed=seed).permutation(nvertices)
    else:
        vertex_order = range(nvertices)

    is_modularity_increasing = True
    while is_modularity_increasing:
        is_modularity_increasing = False
        for vertex in vertex_order:
            best_community, modularity_difference = find_best_community(A, communities, m, modularity, vertex)
            if modularity_difference > 0:
                communities[vertex] = best_community
                modularity += modularity_difference
                is_modularity_increasing = True

    communities = reset_communities(communities)

    print(f"Finished phase1, new modularity: {modularity}")

    return communities, modularity